<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](./data/mountain/art1131.jpg)|![](./data/forest/cdmc317.jpg)|

The problem is realively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several differnet possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D()
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = res.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(res.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

## Load in Data

![skimage-logo](https://scikit-image.org/_static/img/logo.png)

Check out out [`skimage`](https://scikit-image.org/) for useful functions related to processing the images. In particular checkout the documentation for `skimage.io.imread_collection` and `skimage.transform.resize`.

In [2]:
from skimage import io
import os

forests_col = io.imread_collection('./data/forest/*')
mountains_col = io.imread_collection('./data/mountain/*')

forests = forests_col.concatenate()
mountains = mountains_col.concatenate()
forests = forests / 255.0
mountains = mountains / 255.0

print(forests.shape)
print(mountains.shape)
forests[:2]

(328, 256, 256, 3)
(374, 256, 256, 3)


array([[[[0.01176471, 0.        , 0.        ],
         [0.01960784, 0.00392157, 0.        ],
         [0.02352941, 0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.03921569],
         [0.03921569, 0.03921569, 0.07843137],
         [0.        , 0.        , 0.03529412]],

        [[0.01176471, 0.        , 0.        ],
         [0.02745098, 0.00392157, 0.00392157],
         [0.03137255, 0.        , 0.        ],
         ...,
         [0.04313725, 0.04705882, 0.06666667],
         [0.01176471, 0.01568627, 0.03529412],
         [0.        , 0.00392157, 0.02352941]],

        [[0.01960784, 0.        , 0.        ],
         [0.03529412, 0.00392157, 0.        ],
         [0.03921569, 0.        , 0.        ],
         ...,
         [0.        , 0.00392157, 0.01176471],
         [0.        , 0.        , 0.00784314],
         [0.01568627, 0.01568627, 0.02352941]],

        ...,

        [[0.00784314, 0.        , 0.00392157],
         [0.05098039, 0.04313725, 0.04705882]

In [3]:
import numpy as np

X = np.concatenate((forests, mountains), axis=0)
y_ones = np.ones(forests.shape[0])
y_zeroes = np.zeros(mountains.shape[0])
y = np.concatenate((y_ones, y_zeroes), axis=0)
print(X.shape, y.shape)

(702, 256, 256, 3) (702,)


## Instatiate Model

In [4]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

94773248/94765736 [==============================] - 8s 0us/step


In [5]:
for layer in resnet.layers:
    layer.trainable = False

In [9]:
x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='sigmoid')(x)
model = Model(resnet.input, predictions)

In [10]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Fit Model

In [11]:
model.fit(X, y, epochs=5)

Train on 702 samples
Epoch 1/5
702/702 [==============================] - 153s 218ms/sample - loss: 0.1972 - accuracy: 0.9160
Epoch 2/5
702/702 [==============================] - 147s 210ms/sample - loss: 0.0473 - accuracy: 0.9858
Epoch 3/5
702/702 [==============================] - 149s 212ms/sample - loss: 0.0328 - accuracy: 0.9858
Epoch 4/5
702/702 [==============================] - 149s 212ms/sample - loss: 0.1402 - accuracy: 0.9658
Epoch 5/5
702/702 [==============================] - 148s 211ms/sample - loss: 0.1248 - accuracy: 0.9587


In [12]:
scores = model.evaluate(X, y)
print(f'{model.metrics_names[1]}: {scores[1]*100}%')

702/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [13]:
print(f'{model.metrics_names[1]}: {scores[1]*100}%')

accuracy: 52.70655155181885%


# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

In [18]:
X.shape

(702, 256, 256, 3)

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

model_custom = Sequential()
model_custom.add(Conv2D(32, (3,3), activation='relu', input_shape=(256,256,3)))
model_custom.add(MaxPooling2D((2,2)))
model_custom.add(Conv2D(64, (3,3), activation='relu'))
model_custom.add(MaxPooling2D((2,2)))
model_custom.add(Conv2D(64, (3,3), activation='relu'))
model_custom.add(Flatten())
model_custom.add(Dense(64, activation='relu'))
model_custom.add(Dense(2, activation='sigmoid'))

model_custom.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 60, 60, 64)        36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 230400)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)               

In [23]:
# Compile Model
model_custom.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [24]:
# Fit Model
model_custom.fit(X, y, epochs=5)

Train on 702 samples
Epoch 1/5
702/702 [==============================] - 38s 54ms/sample - loss: 1.1698 - accuracy: 0.5670
Epoch 2/5
702/702 [==============================] - 37s 52ms/sample - loss: 0.6931 - accuracy: 0.5328
Epoch 3/5
702/702 [==============================] - 37s 52ms/sample - loss: 0.6931 - accuracy: 0.5328
Epoch 4/5
702/702 [==============================] - 37s 52ms/sample - loss: 0.6931 - accuracy: 0.5328
Epoch 5/5
702/702 [==============================] - 37s 52ms/sample - loss: 0.6931 - accuracy: 0.5328


In [26]:
scores = model_custom.evaluate(X, y)

702/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [27]:
print(f'{model_custom.metrics_names[1]}: {scores[1]*100}%')

accuracy: 53.27635407447815%


### Analysis:

My custom model has (slightly) higher accuracy than the Resnet model!

In [32]:
# trying with 10 epochs

model.fit(X, y, epochs=10)

Train on 702 samples
Epoch 1/10
702/702 [==============================] - 145s 207ms/sample - loss: 0.2209 - accuracy: 0.9772
Epoch 2/10
702/702 [==============================] - 141s 201ms/sample - loss: 0.1610 - accuracy: 0.9772
Epoch 3/10
702/702 [==============================] - 140s 199ms/sample - loss: 0.0066 - accuracy: 0.9986
Epoch 4/10
702/702 [==============================] - 140s 199ms/sample - loss: 0.0197 - accuracy: 0.9957
Epoch 5/10
702/702 [==============================] - 135s 193ms/sample - loss: 0.0211 - accuracy: 0.9957
Epoch 6/10
702/702 [==============================] - 138s 196ms/sample - loss: 0.0977 - accuracy: 0.9858
Epoch 7/10
702/702 [==============================] - 143s 203ms/sample - loss: 0.0086 - accuracy: 0.9972
Epoch 8/10
702/702 [==============================] - 144s 205ms/sample - loss: 0.0131 - accuracy: 0.9972
Epoch 9/10
702/702 [==============================] - 144s 205ms/sample - loss: 0.0275 - accuracy: 0.9900
Epoch 10/10
702/702 [====

In [33]:
scores = model.evaluate(X, y)

702/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [34]:
print(f'{model_custom.metrics_names[1]}: {scores[1]*100}%')

accuracy: 61.68091297149658%


In [35]:
model_custom.fit(X, y, epochs=10)

Train on 702 samples
Epoch 1/10
702/702 [==============================] - 38s 54ms/sample - loss: 0.6931 - accuracy: 0.5328
Epoch 2/10
702/702 [==============================] - 38s 54ms/sample - loss: 0.6931 - accuracy: 0.5328
Epoch 3/10
702/702 [==============================] - 38s 53ms/sample - loss: 0.6931 - accuracy: 0.5328
Epoch 4/10
702/702 [==============================] - 38s 54ms/sample - loss: 0.6931 - accuracy: 0.5328
Epoch 5/10
702/702 [==============================] - 38s 54ms/sample - loss: 0.6931 - accuracy: 0.5328
Epoch 6/10
702/702 [==============================] - 37s 53ms/sample - loss: 0.6931 - accuracy: 0.5328
Epoch 7/10
702/702 [==============================] - 38s 54ms/sample - loss: 0.6931 - accuracy: 0.5328
Epoch 8/10
702/702 [==============================] - 37s 53ms/sample - loss: 0.6931 - accuracy: 0.5328
Epoch 9/10
702/702 [==============================] - 37s 53ms/sample - loss: 0.6931 - accuracy: 0.5328
Epoch 10/10
702/702 [======================

In [36]:
scores = model_custom.evaluate(X, y)

702/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [37]:
print(f'{model_custom.metrics_names[1]}: {scores[1]*100}%')

accuracy: 53.27635407447815%


### Analysis:

With more epochs, the pre-trained model gets better accuracy while my custom model stays about the same.

# Custom CNN Model with Image Manipulations
## *This a stretch goal, and it's relatively difficult*

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

In [ ]:
# State Code for Image Manipulation Here

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language